In [1]:
import warnings
warnings.filterwarnings("ignore")

import env
import Liam_wrangle
import pandas as pd

In [6]:
train = pd.read_csv('train_zillow.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train_zillow.csv'

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
import Liam_explore

In [ ]:
binary_vars = ['bedrooms', 'bathrooms', 'land_type', 'has_heating_system',
               'fips', 'has_fireplace', 'full_bathrooms','has_pool', 'city','county',
               'zip_code','unitcnt','in_los_angeles', 'in_orange_county', 'in_ventura',
                'lot_sqft_bins','acre_bins','square_feet_bins' ]

quant_vars = ['square_feet', 'lot_square_feet', 'quality', 'latitude', 'bathrooms', 'bedrooms',
                'longitude','room_count', 'structure_tax_value','tax_value', 'land_tax_value', 
              'taxamount', 'house_age', 'tax_rate']

categorical_target = 'level_of_log_error'
continuous_target = 'logerror'

In [ ]:
Liam_explore.explore_univariate(train, binary_vars, quant_vars)

## Takeaways

- 6037 = LA
- 6059 = Orange
- 6111 = Ventura

In [ ]:
Liam_explore.explore_bivariate(train, categorical_target, continuous_target, binary_vars, quant_vars)

In [ ]:
import seaborn as sns
from matplotlib.pyplot import figure

figure(figsize=(15, 20), dpi=80)
sns.scatterplot(x = train.longitude, y = train.latitude, hue = train.level_of_log_error)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.subplots(5, 1, figsize=(12,40), sharey=True)
sns.set(style="darkgrid")

plt.subplot(5,1,1)
plt.title("Relationship between location, Bedrooms", size=20, color='black')
sns.scatterplot(data=train, x='longitude', y='latitude', hue=train.bedrooms, palette='viridis')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(5,1,2)
plt.title("Relationship between location, Square Footage of Homes", size=20, color='black')
sns.scatterplot(data=train, x='longitude', y='latitude', hue='square_feet', palette='viridis')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    
plt.subplot(5,1,3)
plt.title("Relationship between location, has_fireplace", size=20, color='black')
sns.scatterplot(data=train, x='longitude', y='latitude', hue=train.has_fireplace, palette='viridis')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(5,1,4)
plt.title("Relationship between location, has_heating_system", size=20, color='black')
sns.scatterplot(data=train, x='longitude', y='latitude', hue=train.has_heating_system, palette='viridis')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(5,1,5)
plt.title("Relationship between location, level_of_log_error", size=20, color='black')
sns.scatterplot(data=train, x='longitude', y='latitude', hue='level_of_log_error', palette='viridis_r')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
train.bathrooms.value_counts()

In [ ]:
sns.pairplot(data = train, hue = 'level_of_log_error', 
             x_vars = ['logerror', 'square_feet_bins', 'lot_sqft_bins', 'acre_bins', 
                       'bedrooms', 'bathrooms'],
             y_vars = ['logerror', 'has_fireplace', 'has_heating_system', 'latitude', 
                       'longitude', 'has_pool'], palette='viridis')

## Takeaways

- The higher the log error the more the home is overvalued compared to the zestimate
- The lower the log error the more the home is undervalued compared to the zestimate

# Clustering

In [ ]:
X = train[['logerror', 'house_age']]
X.head(3)

In [ ]:
# lets plot inertia vs k
from sklearn.cluster import KMeans
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k).fit(X).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

In [ ]:
import viz_kmeans

#define the thing
kmeans = KMeans(n_clusters=4)

# fit the thing
kmeans.fit(X)

# Use (predict using) the thing 
kmeans.predict(X)

In [ ]:
train['cluster'] = kmeans.predict(X)

train.head()

In [ ]:
import numpy as np
# Visualize the scatter plot for Actual vs predicted (K-Means)
viz_zillow(train, kmeans)

In [ ]:
# lets visualize the clusters along with the centers on (scaled data).

plt.figure(figsize=(14, 9))

# scatter plot of data with hue for cluster

sns.scatterplot(x = 'house_age', y= 'square_feet', data = train, hue = train.cluster)

# plot cluster centers (centroids)

centroids_scaled.plot.scatter(x = 'house_age', y = 'square_feet', ax = plt.gca(), color = 'k', alpha = 0.3, s = 500, marker = 'o',)

plt.title('Visualizing Cluster Centers')
plt.legend();